
Neural Networks
===============

使用torch.nn包来构建神经网络。

``nn``包依赖``autograd``包来定义模型并求导。
一个``nn.Module``包含各个层和一个``forward(input)``方法，该方法返回``output``。

神经网络的典型训练过程如下：

1. 定义包含一些可学习的参数(或者叫权重)神经网络模型； 
2. 在数据集上迭代； 
3. 通过神经网络处理输入； 
4. 计算损失(输出结果和正确值的差值大小)；
5. 将梯度反向传播回网络的参数； 
6. 更新网络的参数，主要使用如下简单的更新原则：

``weight = weight - learning_rate * gradient``


例如：

![](https://pytorch.org/tutorials/_images/mnist.png)

它是一个简单的前馈神经网络，它接受一个输入，然后一层接着一层地传递，最后输出计算的结果。

经典的LeNet-5网络

输入：图像Size为3232。这要比mnist数据库中最大的字母(2828)还大。这样做的目的是希望潜在的明显特征，如笔画断续、角点能够出现在最高层特征监测子感受野的中心。

输出：10个类别，分别为0-9数字的概率

* C1层是一个卷积层，有6个卷积核（提取6种局部特征），核大小为5 * 5
* S2层是pooling层，下采样（区域:2 * 2 ）降低网络训练参数及模型的过拟合程度。
* C3层是第二个卷积层，使用16个卷积核，核大小:5 * 5 提取特征
* S4层也是一个pooling层，区域:2*2
* C5层是最后一个卷积层，卷积核大小:5 * 5 卷积核种类:120
* 最后使用全连接层，将C5的120个特征进行分类，最后输出0-9的概率
  

定义网络
------------------

开始定义一个网络：


In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LeNet(nn.Module):
    
    def __init__(self):
        super(LeNet,self).__init__()
        #现在开始构建网络
        self.conv1 = nn.Conv2d(1,6,5) #第一个卷积层
        #一开始输入的是one channel，黑白图像，用六个5*5*1卷积核，输出是six channels
        self.conv2 = nn.Conv2d(6,16,5) #第二个卷积层
        #用了16个5*5*6的卷积核，每一个卷积核会将6 channels压扁（计算），最后变成16 channels
        #self.conv3 = nn.Conv2d(16,120,5) #官方文档没有用到第三个卷积层
        #an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        #Max pooling over a (2,2) window,2*2的窗口取最大值，最后得到的长宽都为1/2
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)),2) #如果窗口为正方形，可以只写一个数字
        #x = F.relu(self.conv3(x))
        x = x.view(-1,self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self,x):
        size = x.size()[1:] #all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = LeNet()
print(net)
        
        

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


上面就定义了神经网络的框架了

* 在模型中必须要定义 ``forward`` 函数,可以在 ``forward`` 函数中使用任何针对 Tensor 的操作。
* ``backward``函数（用来计算梯度）会被``autograd``自动创建。


* ``net.parameters()``返回可被学习的参数（权重）列表和值

In [19]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


测试随机输入32×32。 注：这个网络（LeNet）期望的输入大小是32×32，如果使用MNIST数据集来训练这个网络，请把图片大小重新调整到32×32。

In [20]:
input = torch.randn(1,1,32,32)
#输入的四个数字，即为四维input，NCHW，n_sample * channels * height * width
out = net(input)
print(out)

tensor([[-0.0889, -0.1532,  0.0008, -0.0564, -0.0959,  0.0441, -0.0426, -0.0353,
          0.1194,  0.0079]], grad_fn=<AddmmBackward>)
